# A2. Graph API 取り込み - 実行

## 概要

このノートブックでは、SharePoint から実際にファイルを取得し、Azure AI Search に取り込みます。

### 処理フロー

1. **インデックス作成**: ベクター検索対応のスキーマ定義
2. **ファイル取得**: Graph API でファイル一覧を取得
3. **本文抽出**: PDF, Word, Excel, PowerPoint からテキスト抽出
4. **チャンク分割**: 長文を適切なサイズに分割
5. **埋め込み生成**: Azure OpenAI で埋め込みベクトルを生成 (オプション)
6. **ACL 処理**: ユーザー/グループ権限の取得
7. **アップロード**: Azure AI Search に一括アップロード

### 所要時間

- ファイル数や容量により変動しますが、100ファイル程度で 10〜30分 程度
- 埋め込み生成を有効にすると追加で時間がかかります

## 1. 環境の初期化

In [ ]:
import os
import json
import msal
import requests
import time
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Optional
from tqdm.auto import tqdm

# Azure Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch
)

# ユーティリティ
from utils import (
    TextExtractor,
    TextChunker,
    create_document_id,
    extract_acl_from_permissions
)

# 環境変数の読み込み
load_dotenv(Path("..") / ".env")

# 前回の設定を読み込み
config_path = Path("..") / "config.json"
if config_path.exists():
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)
    print("✅ 前回の設定を読み込みました")
else:
    print("❌ config.json が見つかりません。A1_graph_ingest_setup.ipynb を先に実行してください")
    raise FileNotFoundError("config.json not found")

print(f"\nサイト: {config['site_info']['name']}")
print(f"対象ドライブ数: {len(config['target_drives'])}")
print(f"チャンクサイズ: {config['chunk_size']} 文字")

## 2. Graph API クライアントの準備

In [ ]:
# Graph API クライアント (A1 から再利用)
class GraphAPIClient:
    def __init__(self, tenant_id: str, client_id: str, client_secret: str):
        self.tenant_id = tenant_id
        self.client_id = client_id
        self.authority = f"https://login.microsoftonline.com/{tenant_id}"
        self.base_url = "https://graph.microsoft.com/v1.0"
        
        self.app = msal.ConfidentialClientApplication(
            client_id,
            authority=self.authority,
            client_credential=client_secret
        )
        self.token = None
    
    def get_token(self) -> str:
        result = self.app.acquire_token_for_client(
            scopes=["https://graph.microsoft.com/.default"]
        )
        if "access_token" in result:
            self.token = result["access_token"]
            return self.token
        else:
            raise Exception(f"Token error: {result.get('error_description')}")
    
    def get_headers(self) -> Dict[str, str]:
        if not self.token:
            self.get_token()
        return {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
    
    def get(self, endpoint: str, params: Optional[Dict] = None) -> Dict:
        url = f"{self.base_url}{endpoint}"
        response = requests.get(url, headers=self.get_headers(), params=params)
        
        # リトライロジック (429 対策)
        retry_count = 0
        while response.status_code == 429 and retry_count < 3:
            retry_after = int(response.headers.get("Retry-After", 5))
            print(f"⚠️  Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            response = requests.get(url, headers=self.get_headers(), params=params)
            retry_count += 1
        
        response.raise_for_status()
        return response.json()

# クライアント作成
graph_client = GraphAPIClient(
    tenant_id=os.getenv("GRAPH_TENANT_ID"),
    client_id=os.getenv("GRAPH_CLIENT_ID"),
    client_secret=os.getenv("GRAPH_CLIENT_SECRET")
)

print("✅ Graph API クライアントを初期化しました")

## 3. Azure AI Search インデックスの作成

In [ ]:
# Search クライアント
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(search_key)
)

# ベクター検索を使うかどうか
use_vector_search = bool(os.getenv("AZURE_OPENAI_ENDPOINT"))
vector_dimensions = 3072  # text-embedding-ada-002 のデフォルト

if use_vector_search:
    print(f"✅ ベクター検索を有効化します (次元数: {vector_dimensions})")
else:
    print("⚠️  Azure OpenAI が設定されていないため、ベクター検索は無効です")

print(f"\nインデックス名: {index_name}")

In [ ]:
# インデックススキーマの定義
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SimpleField(name="url", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="path", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="site", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="library", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="contentType", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SimpleField(name="fileExtension", type=SearchFieldDataType.String, filterable=True, facetable=True),
    SimpleField(name="lastModified", type=SearchFieldDataType.DateTimeOffset, sortable=True, filterable=True),
    SimpleField(name="createdBy", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="modifiedBy", type=SearchFieldDataType.String, filterable=True),
    SimpleField(name="size", type=SearchFieldDataType.Int64, sortable=True, filterable=True),
    SimpleField(name="chunkIndex", type=SearchFieldDataType.Int32, filterable=True),
    SimpleField(name="aclUsers", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True),
    SimpleField(name="aclGroups", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True),
]

# ベクター検索設定
vector_search_config = None
if use_vector_search:
    # ベクターフィールドを追加
    fields.append(
        SearchField(
            name="contentVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=vector_dimensions,
            vector_search_profile_name="default-vector-profile"
        )
    )
    
    # ベクター検索設定
    vector_search_config = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="default-hnsw",
                parameters={
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="default-vector-profile",
                algorithm_configuration_name="default-hnsw"
            )
        ]
    )

# セマンティック検索設定
semantic_config = SemanticConfiguration(
    name="default",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        content_fields=[SemanticField(field_name="content")]
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])

# インデックスの作成
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search_config,
    semantic_search=semantic_search
)

try:
    # 既存のインデックスを削除 (オプション)
    try:
        index_client.delete_index(index_name)
        print(f"⚠️  既存のインデックス '{index_name}' を削除しました")
        time.sleep(2)  # 削除完了を待つ
    except:
        pass
    
    # 新規作成
    result = index_client.create_index(index)
    print(f"\n✅ インデックスを作成しました: {result.name}")
    print(f"   フィールド数: {len(result.fields)}")
    if use_vector_search:
        print(f"   ベクター検索: 有効 ({vector_dimensions} 次元)")
    print(f"   セマンティック検索: 有効")
    
except Exception as e:
    print(f"❌ エラー: {e}")
    raise

## 4. Azure OpenAI クライアントの準備 (オプション)

In [ ]:
openai_client = None
embedding_deployment = None

if use_vector_search:
    try:
        from openai import AzureOpenAI
        
        openai_client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
        )
        embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
        
        # テスト
        test_response = openai_client.embeddings.create(
            input="test",
            model=embedding_deployment
        )
        vector_dimensions = len(test_response.data[0].embedding)
        
        print(f"✅ Azure OpenAI クライアントを初期化しました")
        print(f"   デプロイメント: {embedding_deployment}")
        print(f"   埋め込み次元: {vector_dimensions}")
        
    except Exception as e:
        print(f"⚠️  Azure OpenAI の初期化に失敗: {e}")
        print("   ベクター検索なしで続行します")
        use_vector_search = False
else:
    print("⚠️  ベクター検索は無効です")

## 5. ファイル取得と処理

In [ ]:
def get_all_files_recursive(
    graph_client: GraphAPIClient,
    drive_id: str,
    folder_id: str = "root",
    supported_extensions: List[str] = None,
    max_files: int = 100
) -> List[Dict]:
    """
    ドライブ内のファイルを再帰的に取得
    """
    all_files = []
    
    def traverse(item_id: str, current_path: str = ""):
        if len(all_files) >= max_files:
            return
        
        try:
            endpoint = f"/drives/{drive_id}/items/{item_id}/children"
            response = graph_client.get(endpoint, params={"$top": 200})
            
            for item in response.get("value", []):
                if len(all_files) >= max_files:
                    break
                
                item_name = item["name"]
                item_path = f"{current_path}/{item_name}" if current_path else item_name
                
                if "folder" in item:
                    # フォルダの場合は再帰
                    traverse(item["id"], item_path)
                elif "file" in item:
                    # ファイルの場合
                    ext = Path(item_name).suffix.lower()
                    
                    if supported_extensions is None or ext in supported_extensions:
                        file_info = {
                            "id": item["id"],
                            "name": item_name,
                            "path": item_path,
                            "extension": ext,
                            "size": item.get("size", 0),
                            "webUrl": item.get("webUrl", ""),
                            "lastModified": item.get("lastModifiedDateTime", ""),
                            "createdBy": item.get("createdBy", {}).get("user", {}).get("displayName", ""),
                            "modifiedBy": item.get("lastModifiedBy", {}).get("user", {}).get("displayName", ""),
                        }
                        all_files.append(file_info)
        
        except Exception as e:
            print(f"⚠️  フォルダ {current_path} の探索でエラー: {str(e)[:100]}")
    
    traverse(folder_id)
    return all_files

# 全ドライブからファイルを取得
all_files = []
supported_exts = config["supported_extensions"]
max_files = config["max_files"]

print("ファイルを取得しています...\n")

for drive in config["target_drives"]:
    print(f"📁 {drive['name']}")
    files = get_all_files_recursive(
        graph_client,
        drive["id"],
        "root",
        supported_exts,
        max_files - len(all_files)
    )
    
    for f in files:
        f["drive_id"] = drive["id"]
        f["drive_name"] = drive["name"]
    
    all_files.extend(files)
    print(f"   取得: {len(files)} 件\n")
    
    if len(all_files) >= max_files:
        break

print("=" * 60)
print(f"✅ 合計 {len(all_files)} 件のファイルを取得しました")
print("=" * 60)

# 統計
total_size = sum(f["size"] for f in all_files) / (1024 * 1024)
print(f"\n合計サイズ: {total_size:.2f} MB")

# 拡張子別集計
ext_count = {}
for f in all_files:
    ext = f["extension"]
    ext_count[ext] = ext_count.get(ext, 0) + 1

print("\n拡張子別:")
for ext, count in sorted(ext_count.items()):
    print(f"  {ext}: {count} 件")

## 6. ファイル処理とドキュメント生成

In [ ]:
def process_file(
    graph_client: GraphAPIClient,
    file_info: Dict,
    site_info: Dict,
    chunker: TextChunker,
    openai_client = None,
    embedding_deployment: str = None
) -> List[Dict]:
    """
    1つのファイルを処理してドキュメントリストを返す
    """
    try:
        # ファイルコンテンツのダウンロード
        download_url = f"/drives/{file_info['drive_id']}/items/{file_info['id']}/content"
        content_response = requests.get(
            f"{graph_client.base_url}{download_url}",
            headers=graph_client.get_headers(),
            allow_redirects=True
        )
        content_response.raise_for_status()
        content_bytes = content_response.content
        
        # テキスト抽出
        text = TextExtractor.extract(content_bytes, file_info["extension"])
        
        if not text or len(text.strip()) < 10:
            return []  # 空のファイルはスキップ
        
        # ACL取得 (本番環境実装)
        acl_users = []
        acl_groups = []
        
        try:
            # Permissions APIを呼び出してアクセス権限を取得
            permissions_endpoint = f"/drives/{file_info['drive_id']}/items/{file_info['id']}/permissions"
            permissions_response = graph_client.get(permissions_endpoint)
            permissions = permissions_response.get("value", [])
            
            # ACLを抽出
            acl_users, acl_groups = extract_acl_from_permissions(permissions)
            
        except Exception as e:
            # Permission取得に失敗しても処理を続行
            print(f"⚠️  Permission取得エラー ({file_info['name']}): {str(e)[:80]}")
        
        # チャンク分割
        chunks = chunker.split_text(text, metadata=file_info)
        
        # ドキュメント生成
        documents = []
        for chunk in chunks:
            doc_id = create_document_id(
                site_info["id"],
                file_info["drive_id"],
                file_info["id"],
                chunk.chunk_index
            )
            
            doc = {
                "id": doc_id,
                "title": file_info["name"],
                "content": chunk.text,
                "url": file_info["webUrl"],
                "path": file_info["path"],
                "site": site_info["name"],
                "library": file_info["drive_name"],
                "contentType": file_info["extension"][1:] if file_info["extension"] else "unknown",
                "fileExtension": file_info["extension"],
                "lastModified": file_info["lastModified"],
                "createdBy": file_info["createdBy"],
                "modifiedBy": file_info["modifiedBy"],
                "size": file_info["size"],
                "chunkIndex": chunk.chunk_index,
                "aclUsers": acl_users,
                "aclGroups": acl_groups,
            }
            
            # 埋め込み生成 (オプション)
            if openai_client and embedding_deployment:
                try:
                    response = openai_client.embeddings.create(
                        input=chunk.text[:8000],  # トークン制限対策
                        model=embedding_deployment
                    )
                    doc["contentVector"] = response.data[0].embedding
                except Exception as e:
                    print(f"⚠️  埋め込み生成エラー ({file_info['name']}): {str(e)[:50]}")
            
            documents.append(doc)
        
        return documents
    
    except Exception as e:
        print(f"❌ ファイル処理エラー ({file_info['name']}): {str(e)[:100]}")
        return []

# チャンカーの準備
chunker = TextChunker(
    chunk_size=config["chunk_size"],
    chunk_overlap=config["chunk_overlap"]
)

print("✅ ファイル処理の準備が完了しました")
print(f"   チャンクサイズ: {config['chunk_size']} 文字")
print(f"   オーバーラップ: {config['chunk_overlap']} 文字")

In [ ]:
# 全ファイルを処理
all_documents = []
failed_files = []

print("\nファイルを処理しています...\n")
print("=" * 60)

for file_info in tqdm(all_files, desc="処理中"):
    docs = process_file(
        graph_client,
        file_info,
        config["site_info"],
        chunker,
        openai_client,
        embedding_deployment
    )
    
    if docs:
        all_documents.extend(docs)
    else:
        failed_files.append(file_info["name"])
    
    # API制限対策: 少し待つ
    if use_vector_search:
        time.sleep(0.1)

print("\n" + "=" * 60)
print(f"✅ 処理完了")
print(f"   成功: {len(all_files) - len(failed_files)} / {len(all_files)} ファイル")
print(f"   生成ドキュメント: {len(all_documents)} 件")
if failed_files:
    print(f"   失敗: {len(failed_files)} 件")
    print(f"   失敗ファイル: {failed_files[:5]}" + (" ..." if len(failed_files) > 5 else ""))
print("=" * 60)

## 7. Azure AI Search へのアップロード

In [ ]:
# Search クライアント
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

# 既存のドキュメントをクリア (オプション)
print("既存のドキュメントをクリアしています...")

try:
    # インデックス内の全ドキュメントを検索
    search_results = search_client.search(
        search_text="*",
        top=1000,  # 一度に取得する最大数
        select=["id"]
    )
    
    existing_docs = [{"id": doc["id"]} for doc in search_results]
    
    if existing_docs:
        print(f"既存ドキュメント {len(existing_docs)} 件を削除中...")
        
        # バッチ削除
        batch_size = 1000
        for i in range(0, len(existing_docs), batch_size):
            batch = existing_docs[i:i + batch_size]
            try:
                search_client.delete_documents(documents=batch)
                print(f"  削除済み: {min(i + batch_size, len(existing_docs))} / {len(existing_docs)}")
            except Exception as e:
                print(f"  削除エラー: {str(e)[:100]}")
        
        # 削除完了を待つ
        time.sleep(3)
        print("✅ 既存ドキュメントの削除が完了しました")
    else:
        print("削除対象のドキュメントはありません")

except Exception as e:
    print(f"⚠️  ドキュメント削除でエラー: {str(e)[:100]}")
    print("   新しいドキュメントを追加します")

In [ ]:
# バッチアップロード
batch_size = 100  # 1回のアップロードで送信するドキュメント数
total_uploaded = 0
total_failed = 0

print("\nAzure AI Search にアップロードしています...\n")
print("=" * 60)

for i in tqdm(range(0, len(all_documents), batch_size), desc="アップロード中"):
    batch = all_documents[i:i + batch_size]
    
    try:
        result = search_client.upload_documents(documents=batch)
        
        # 結果の確認
        for res in result:
            if res.succeeded:
                total_uploaded += 1
            else:
                total_failed += 1
                print(f"⚠️  アップロード失敗: {res.key} - {res.error_message}")
    
    except Exception as e:
        total_failed += len(batch)
        print(f"❌ バッチアップロードエラー: {str(e)[:100]}")
    
    # スロットリング対策
    time.sleep(0.5)

print("\n" + "=" * 60)
print("✅ アップロード完了")
print(f"   成功: {total_uploaded} / {len(all_documents)} ドキュメント")
if total_failed > 0:
    print(f"   失敗: {total_failed} ドキュメント")
print("=" * 60)

## 8. インデックスの確認

In [ ]:
# インデックス統計
time.sleep(3)  # インデックスの更新を待つ

try:
    # 簡単なカウントクエリ
    results = search_client.search(
        search_text="*",
        top=1,
        include_total_count=True
    )
    
    total_count = results.get_count()
    
    print("\n" + "=" * 60)
    print("📊 インデックス統計")
    print("=" * 60)
    print(f"インデックス名: {index_name}")
    print(f"ドキュメント数: {total_count:,} 件")
    print(f"\n処理結果:")
    print(f"  処理ファイル: {len(all_files)} 件")
    print(f"  生成ドキュメント: {len(all_documents)} 件")
    print(f"  平均チャンク数/ファイル: {len(all_documents) / max(len(all_files), 1):.1f}")
    print("=" * 60)
    
except Exception as e:
    print(f"⚠️  統計取得エラー: {e}")

## まとめ

このノートブックで以下を完了しました:

✅ Azure AI Search インデックスの作成  
✅ SharePoint からファイルを取得  
✅ テキスト抽出とチャンク分割  
✅ 埋め込みベクトルの生成 (オプション)  
✅ Azure AI Search へのアップロード  

### 次のステップ

**A3_graph_delta_sync.ipynb** に進んで、差分同期の仕組みを学びます。

### トラブルシューティング

**アップロードエラー**:
- ドキュメントIDが重複していないか確認
- ペイロードサイズが 16MB を超えていないか確認

**埋め込み生成エラー**:
- Azure OpenAI のクォータを確認
- 入力テキストが 8192 トークンを超えていないか確認

**API スロットリング**:
- `time.sleep()` の値を調整
- バッチサイズを小さくする

### パフォーマンスチューニング

- **並列処理**: `concurrent.futures` で複数ファイルを同時処理
- **バッチサイズ**: ネットワークとメモリに応じて調整
- **チャンクサイズ**: 検索精度と処理速度のバランス